In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc, classification_report
import seaborn as sns

# Load and preprocess the data
def load_fashion_mnist():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.
    x_train = x_train[..., tf.newaxis]
    x_test = x_test[..., tf.newaxis]
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    return x_train, y_train, x_test, y_test

# Simplified Capsule Layer (Placeholder for actual implementation)
def CapsuleLayer(inputs, num_capsule, dim_capsule):
    # Reshape into [None, num_capsules, dim_capsule]
    x = layers.Dense(num_capsule * dim_capsule)(inputs)
    x = layers.Reshape((num_capsule, dim_capsule))(x)
    # Summing over the last dimension to simulate capsule output (not a true capsule implementation)
    return layers.Lambda(lambda s: tf.reduce_sum(s, axis=-1))(x)

# Build CapsNet model
def CapsNet(input_shape):
    x_input = layers.Input(shape=input_shape)
    conv1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu')(x_input)
    flat = layers.Flatten()(conv1)
    primary_caps = CapsuleLayer(flat, num_capsule=10, dim_capsule=16)
    output = layers.Activation('softmax')(primary_caps)  # Use softmax for classification
    model = models.Model(inputs=x_input, outputs=output)
    return model

# Training the model
def train_model(model, x_train, y_train, x_test, y_test):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))
    return history

# Plotting confusion matrix
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt="d", linewidths=.5)
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Plotting ROC curves
def plot_roc_curve(y_test, y_pred):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(10):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure(figsize=(8, 6))
    for i in range(10):
        plt.plot(fpr[i], tpr[i], label='Class %i (area = %0.2f)' % (i, roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()

# Main function to run the script
if __name__ == "__main__":
    x_train, y_train, x_test, y_test = load_fashion_mnist()
    model = CapsNet(x_train.shape[1:])
    history = train_model(model, x_train, y_train, x_test, y_test)
    y_pred = model.predict(x_test)
    y_pred_label = np.argmax(y_pred, axis=1)
    y_true_label = np.argmax(y_test, axis=1)
    plot_confusion_matrix(y_true_label, y_pred_label)
    plot_roc_curve(y_test, y_pred)
    print(classification_report(y_true_label, y_pred_label))
